In [1]:
from main import load_edgelist
from typing import List, Dict
from paco import TimeStampedLinkList, paco
from time import time
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from baseline import create_temporal_network, get_total_causal_paths
import pandas as pd

In [2]:
dataset = pd.read_csv('email-Eu-core-temporal.txt', delimiter=' ', header=None)
dataset.columns = ['source', 'target', 'timestamp']
edge_list = [(row['source'], row['target'], int(row['timestamp'])) for _, row in dataset.iterrows()]

In [3]:
data = TimeStampedLinkList.from_edgelist(edge_list)

In [4]:
delta = 30 #seconds

## PaCo

In [5]:
def execute_paco(delta, k):
    start = time()
    C = paco(data, delta, k)
    end = time()
    
    count = sum(C.values())
    return count, end-start

In [6]:
results = []
for k in trange(1, 6):
    count, runtime = execute_paco(delta, k)
    results.append((count, runtime, k))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.36it/s]


In [7]:
_, paco_runtimes_sec, paco_k = zip(*results)

## Baseline

In [8]:
t = create_temporal_network(edge_list)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 332334/332334 [00:04<00:00, 78599.25it/s]


In [ ]:
results = []
for k in trange(1, 6):
    count, runtime = get_total_causal_paths(t, delta, k)
    results.append((count, runtime, k))

  0%|                                                                                                                                                                                         | 0/5 [00:00<?, ?it/s]

2023-12-18 00:26:55 [Severity.INFO]	Constructing time-unfolded DAG ...
2023-12-18 00:29:21 [Severity.INFO]	finished.
Directed Acyclic Graph
Nodes:		9922811
Roots:		208485
Leaves:		9713303
Links:		9820080
Acyclic:	None

2023-12-18 00:29:21 [Severity.INFO]	Generating causal trees for 208485 root nodes ...
2023-12-18 00:29:26 [Severity.INFO]	Analyzing tree 41697/208485 ...
2023-12-18 00:29:31 [Severity.INFO]	Analyzing tree 83394/208485 ...
2023-12-18 00:29:36 [Severity.INFO]	Analyzing tree 125091/208485 ...
2023-12-18 00:29:41 [Severity.INFO]	Analyzing tree 166788/208485 ...
2023-12-18 00:29:46 [Severity.INFO]	Analyzing tree 208485/208485 ...
2023-12-18 00:29:46 [Severity.INFO]	finished.


 20%|███████████████████████████████████▏                                                                                                                                            | 1/5 [02:53<11:35, 173.96s/it]

Number of causal paths: Total path count: 		326333.0 
[Unique / Sub paths / Total]: 	[26109.0 / 657204.0 / 983537.0]
Nodes:				986 
Edges:				24929
Max. path length:		7
Avg path length:		1.0045658882184763 
Paths of length k = 0		0.0 [ 0.0 / 654156.0 / 654156.0 ]
Paths of length k = 1		324892.0 [ 24868.0 / 2931.0 / 327823.0 ]
Paths of length k = 2		1401.0 [ 1211.0 / 89.0 / 1490.0 ]
Paths of length k = 3		37.0 [ 27.0 / 12.0 / 49.0 ]
Paths of length k = 4		0.0 [ 0.0 / 9.0 / 9.0 ]
Paths of length k = 5		1.0 [ 1.0 / 5.0 / 6.0 ]
Paths of length k = 6		1.0 [ 1.0 / 2.0 / 3.0 ]
Paths of length k = 7		1.0 [ 1.0 / 0.0 / 1.0 ]


2023-12-18 00:29:49 [Severity.INFO]	Constructing time-unfolded DAG ...


In [ ]:
_, baseline_runtimes_sec, baseline_k = zip(*results)

## Plots

In [ ]:
plt.plot(paco_k, paco_runtimes_sec, 's--', color='blue',label="PaCo")
plt.plot(baseline_k, baseline_runtimes_sec, '^--', color='red',label="Baseline")
plt.xlabel("K")
plt.ylabel("run-time [sec]")
plt.grid(True)
plt.legend()
plt.savefig('plots/email_right_panel.png')
plt.show()